In [1]:
import pandas as pd
from datetime import timedelta

In [5]:
df = pd.read_csv("/Users/naveenapaleti/Projects/ShopTrack360/data/cleaned_ecommerce_data.csv")
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]

In [3]:
latest_date = df['InvoiceDate'].max()
churn_threshold = 30  # days
cutoff_date = latest_date - timedelta(days=churn_threshold)

In [6]:
customer_df = df.groupby('CustomerID').agg({
    'InvoiceDate': [lambda x: (latest_date - x.max()).days,  # Recency
                    lambda x: (x.max() - x.min()).days + 1], # Engagement span
    'InvoiceNo': 'nunique',     # Frequency
    'TotalPrice': 'sum',        # Monetary
    'StockCode': 'nunique',     # Product diversity
    'Quantity': 'sum',          # Basket size
    'Country': 'first'          # Country
}).reset_index()

In [7]:
customer_df.columns = [
    'CustomerID', 'Recency', 'EngagementDays',
    'Frequency', 'Monetary', 'UniqueProducts',
    'TotalQuantity', 'Country'
]

In [8]:
customer_df['AvgBasketSize'] = customer_df['TotalQuantity'] / customer_df['Frequency']
customer_df['AvgSpendPerTxn'] = customer_df['Monetary'] / customer_df['Frequency']

In [9]:
# Last purchase date per customer
last_purchase = df.groupby('CustomerID')['InvoiceDate'].max()
churned_customers = last_purchase[last_purchase < cutoff_date].index

customer_df['ChurnLabel'] = customer_df['CustomerID'].isin(churned_customers).astype(int)

In [10]:
customer_df.to_csv("/Users/naveenapaleti/Projects/ShopTrack360/data/customer_features.csv", index=False)